In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

#retry helper
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

#set up google api key
# Install kaggle package if not already installed
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("GOOGLE_API_KEY is not set. Set it as an environment variable.")


In [6]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain AI to me like I'm an old lady.")

Markdown(response.text)

Alright, dearie, let's talk about this "AI" thing. Think of it like this:

Imagine you have a very, very smart parrot. Not just one that repeats words, but one that can learn things, like how to play a simple game. You feed it information – "if you peck the blue block, you get a cracker" – and the parrot learns. The more you teach it, the better it gets at playing the game.

AI is kind of like that parrot, but instead of a parrot, it's a computer program. And instead of you teaching it with crackers, you teach it with lots and lots of data – information, pictures, words, whatever you want it to learn.

So, you show this program a million pictures of cats and tell it, "These are cats!"  Then, you show it a new picture and ask, "Is this a cat?" If it's learned well enough, it can say, "Yes, that's a cat!"

That's the basic idea. We feed these computer programs tons of information, and they learn to recognize patterns and make decisions based on those patterns.

**Think of some examples you might know:**

*   **When you ask Siri or Alexa a question:**  You're talking to an AI program that's learned to understand spoken words and search for information.

*   **When Netflix recommends a movie:**  It's using AI to figure out what movies you might like based on what you've watched before.

*   **When your bank flags a suspicious transaction on your credit card:**  That's AI looking for unusual spending patterns.

**Now, here's the important thing to remember:**

AI isn't magic. It's just really complicated computer programs.  It's not *thinking* like a person, it's just *processing* information very quickly and efficiently.

It can be helpful, like finding information or suggesting a good movie. It can also be a little scary, like when we worry about computers taking over jobs. But mostly, it's just a tool, like a washing machine or a microwave. It can make our lives easier if we use it wisely.

Does that make sense, dearie?  Just a clever parrot, learning from lots and lots of data.


In [ ]:
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('Hello! My name is Devisha.')
print(response.text)

Hello Devisha! It's nice to meet you. How can I help you today?



In [9]:
response = chat.send_message('Do you remember what my name is?')
print(response.text)

Yes, your name is Devisha.



In [10]:
#choose the model
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking

In [11]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-1.5-pro':
    pprint(model.to_json_dict())
    break

{'description': 'Stable version of Gemini 1.5 Pro, our mid-size multimodal '
                'model that supports up to 2 million tokens, released in May '
                'of 2024.',
 'display_name': 'Gemini 1.5 Pro',
 'input_token_limit': 2000000,
 'name': 'models/gemini-1.5-pro',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '001'}


In [12]:
#output length
from google.genai import types

short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write 10 words about olives.')

print(response.text)

Salty, briny, green, black, Mediterranean, tapenade, oil, pizza, appetizer, flavorful.



In [15]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write a short poem on RGIPT in 50words.')

print(response.text)

In RGIPT's halls, knowledge thrives,
Petroleum's future, students strive.
Innovation blooms, with minds so bright,
Fueling progress, day and night.
A legacy forged, in fields so vast,
RGIPT shines, forever to last.



In [20]:
low_temp_config = types.GenerateContentConfig(temperature=0.0)

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=low_temp_config,
      contents='Pick a random colour... (respond in a single word)')

  if response.text:
    print(response.text, '-' * 25)

Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------


In [22]:
#top-p
model_config = types.GenerateContentConfig(
    # These are the default values for gemini-2.0-flash.
    temperature=1.0,
    top_p=0.95,
)

story_prompt = "You are a creative writer. Write a short story in 100words about a cat who goes on an adventure."
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=story_prompt)

print(response.text)

Jasper, a ginger tabby with emerald eyes, dreamed of more than sunbeams and catnip. One day, the back door lay ajar. Freedom! He slipped out, the scent of damp earth and blooming honeysuckle intoxicating. 

He stalked butterflies in fields of daisies, chased squirrels up ancient oaks, and lapped water from a babbling brook. The world buzzed with new sounds and smells. As dusk painted the sky, a lonely owl hooted. Suddenly, Jasper felt a pang of hunger and a whisper of fear. He missed his warm bed. Following the familiar scent of home, he padded back, a changed cat, his heart filled with the wild song of adventure.



In [23]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5,
)

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt)

print(response.text)

POSITIVE



In [24]:
#enum mode: The models are trained to generate text, 
# and while the Gemini 2.0 models are great at following instructions, 
# other models can sometimes produce more text than you may wish for.
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt)

print(response.text)

positive


In [26]:
#convert to json model
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])

print(response.text)

import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="Can I have a large dessert pizza with apple and chocolate")

print(response.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```

{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert"
}


In [28]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

Markdown(response.text)

Here's how to solve the problem step-by-step:

1.  **Find the age difference:** When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old.

2.  **Calculate the age gap:** The age difference between you and your partner is 12 - 4 = 8 years.

3.  **Determine the partner's current age:** Since the age difference remains constant, your partner is always 8 years older than you. Now that you are 20, your partner is 20 + 8 = 28 years old.

**Therefore, your partner is 28 years old.**

In [32]:
#code prompting
# The Gemini models love to talk, so it helps to specify they stick to the code if that
# is all that you want.
code_prompt = """
Write a Python code for depth first search. No explanation, provide only the code and output.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ),
    contents=code_prompt)

Markdown(response.text)

```python
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    print(start, end=" ")

    for neighbor in graph[start]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited)

# Example graph
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

print("DFS traversal starting from A:")
dfs(graph, 'A')
print()
```

```
DFS traversal starting from A:
A B D E F C 
```

In [33]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## Status {part.code_execution_result.outcome}'))

        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

```python
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    print(start, end=" ")

    for neighbor in graph[start]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited)

# Example graph
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

print("DFS traversal starting from A:")
dfs(graph, 'A')
print()
```

```
DFS traversal starting from A:
A B D E F C 
```

In [34]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## Status {part.code_execution_result.outcome}'))

        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

```python
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    print(start, end=" ")

    for neighbor in graph[start]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited)

# Example graph
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

print("DFS traversal starting from A:")
dfs(graph, 'A')
print()
```

```
DFS traversal starting from A:
A B D E F C 
```

In [35]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=explain_prompt)

Markdown(response.text)

This file is a **bash script** designed to enhance your command-line prompt with **Git repository status information**.  It aims to make your prompt display details about the current Git branch, the status of your working directory (e.g., changes, staged files, untracked files), and even the relationship to remote branches (ahead, behind).

Here's a breakdown of why you would use it:

*   **Git Awareness:** You get immediate visual feedback on the Git status of the directory you're in directly in your prompt. This eliminates the need to constantly run `git status` to check for changes.
*   **Informative Prompt:** The prompt can display the current branch, whether you have uncommitted changes, if you're ahead or behind the remote branch, and other relevant Git information.
*   **Customization:**  The script provides mechanisms for customizing the appearance of the prompt via themes and variable settings, including colors and symbols.
*   **Efficiency:** By displaying Git status directly in the prompt, it improves your workflow when working with Git repositories.

In essence, this script automates the process of displaying useful Git information in your command-line prompt, making it easier to keep track of your Git repository's status as you work.
